# Create PGW WRF intermediate files

In [7]:
import os, glob, sys
import xarray as xr
import pywinter.winter as pyw

In [8]:
base_dir            = '/Users/doan/Google Drive/My Drive/share/2024/PGW_DS/'
run_id              = 'tok_test_dom'
src_dir             = base_dir + "/Run_WRF/"+run_id+"control"
dst_dir             = base_dir + "/Run_WRF/"+run_id+"/PGW_DS"
wrf_inter_prefix    = 'ERA'

In [9]:
aa  = '1990-2019_2070-2099'
g   = 'EC-Earth3'
ssp = 'ssp585'
gwi_tdom_dir = base_dir + 'GWI4TDOM/' + run_id + '/'
gwi_file = gwi_tdom_dir + '/'+ ssp + '.' + g + '.' + aa + '.nc'
dg = xr.open_dataset(gwi_file)
ff = sorted(glob.glob(src_dir+'/'+wrf_inter_prefix+'*'))

## Read intermediate and add GWI on these files using pywinter library

    #print(interfile.keys())
    #print(interfile['TT'].general)
    #print(interfile['TT'].geoinfo)
    ##print(interfile['TT'].val)
    #print(interfile['TT'].val.shape)

In [10]:
for f in ff[:]:
    print(f)    
    prefix = os.path.basename(f).split(':')[0]
    date = f.split(':')[1]
    print(date)
    imon = int(date.split('-')[1])
    
    infile = src_dir+'/'+prefix+':' + date
    interfile = pyw.rinter(infile)

    var = list(interfile.keys())
    
    x = interfile[var[0]]
    slat, slon = x.geoinfo['STARTLAT'], x.geoinfo['STARTLON']
    dlat, dlon = x.geoinfo['DELTALAT'], x.geoinfo['DELTALON']
    winter_geo = pyw.Geo0(slat,slon,dlat,dlon)

    total_fields = []
    
    for v in var:
        x = interfile[v]
        # Here to put data on renalysis ***
        if v in ['TT', 'UU', 'VV', 'GHT', 'SKINTEMP', 'SST']:
            print('*** Add values on: ', v)
            gwi_val = dg[v].sel(month = imon).values
            values = x.val + gwi_val
        
        else: values = x.val
            
        #=======================
        # Here to write intermediate
        if v in ['SM', 'ST']: # soil variables
            sl_layer = x.level
            winter = pyw.Vsl(v,values,sl_layer)
        elif v in ['RH', 'TT', 'UU', 'VV', 'GHT']: # possible error in future if more 3-D variables exist
            winter = pyw.V3dp(v,values,x.level)
        else:
            winter = pyw.V2d(v,values,x.general['DESC'],x.general['UNITS'], x.general['XLVL'])
        total_fields.append(winter)

    pyw.cinter(prefix,date, winter_geo, total_fields, dst_dir)
    

/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ERA5A:2023-01-01_00
2023-01-01_00
*** Add values on:  GHT
*** Add values on:  VV
*** Add values on:  TT
*** Add values on:  UU
ERA5A:2023-01-01_00
/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ERA5A:2023-01-01_03
2023-01-01_03
*** Add values on:  GHT
*** Add values on:  VV
*** Add values on:  TT
*** Add values on:  UU
ERA5A:2023-01-01_03
/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ERA5A:2023-01-01_06
2023-01-01_06
*** Add values on:  GHT
*** Add values on:  VV
*** Add values on:  TT
*** Add values on:  UU
ERA5A:2023-01-01_06
/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ERA5A:2023-01-01_09
2023-01-01_09
*** Add values on:  GHT
*** Add values on:  VV
*** Add values on:  TT
*** Add values on:  UU
ERA5A:2023-01-01_09
/Users/doan/Google Drive/My Drive/share/2024/PGW_DS//Run_WRF/control/ERA5A:2023-01-01_12
2023-01-01_12
*** Add values on:  GHT
*** Add v